In [1]:
import pandas as pd 
import ast

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import tqdm

In [3]:
def read_list_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        # Supprime les retours à la ligne
        lines = [line.strip() for line in file if line.strip()]
    return lines
data_old = read_list_from_file('/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/a.json')
columns = [col.strip('` ') for col in data_old[0].strip('(),').split(',')]
data = [ ast.literal_eval(data_element.rstrip(',')) for data_element in data_old[1:]]

In [15]:
all_old_data = pd.DataFrame(columns=columns, data=data)
all_dispo_code = pd.read_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/Medicament.csv").dropna()

In [16]:
all_old_data["Code_EAN_13"]= all_old_data["Code_EAN_13"].astype(str)
all_dispo_code['Code_EAN_13']= all_dispo_code['Code_EAN_13'].astype(int)
all_dispo_code['Code_EAN_13']= all_dispo_code['Code_EAN_13'].astype(str)

In [17]:
all_dispo_code['Code_EAN_13']

0       6118000420422
1       6118001072330
2       6118001072347
3       6118000340669
4       6118001170388
            ...      
3233    6118000020172
3234    6118000020165
3235    6118000020158
3236    6118001184514
3237    6118001180813
Name: Code_EAN_13, Length: 3238, dtype: object

In [18]:
all_dispo_code['Code_EAN_13'] = all_dispo_code['Code_EAN_13'].astype(int)

In [19]:
resultat = all_old_data[~all_old_data['Code_EAN_13'].isin(all_dispo_code['Code_EAN_13'])]

In [20]:
resultat

,ID_Medicament,Nom,Caracteristique,Code_EAN_13,Medicament_Generique,Prix_Officine,Prix_Public_De_Vente,Prix_Base_Remboursement,Prix_Hospitalier,Substance_Active_DCI,Classe_Therapeutique,Min_Stock,Stock_Actuel
0,1,URO / EAU POUR IRRIGATION,1 POCHE 3 L - 3000 ML,6118001230068,P,95.0,95.0,95.0,0.0,EAU POUR PREPARATION INJECTABLE,SOLUTION POUR IRRIGATION,0,0
1,2,ELOXATINE 5 MG/ML,1 BOITE 1 FLACON 40 ML - 200 MG,6118010116230,P,2882.0,2882.0,2882.0,2555.0,OXALIPLATINE,SOLUTION A DILUER POUR PERFUSION,0,0
2,3,ELOXATINE 5 MG/ML,1 BOITE 1 FLACON 40 ML - 200 MG,6118001081646,P,2882.0,2882.0,2882.0,2555.0,OXALIPLATINE,SOLUTION A DILUER POUR PERFUSION,0,0
3,4,VIVALAN,1 BOITE 20 COMPRIME - 100 MG,6118001182459,P,60.7,60.7,60.7,40.1,VILOXAZINE,COMPRIME PELLICULE,0,0
4,5,ZADRYL,1 FLACON 60 ML - 1 MG,6118000120155,G,31.9,31.9,31.9,19.9,CETIRIZINE,SOLUTION BUVABLE,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,6110,DOLICOX 120 MG / 7 COMPRIMES,7 COMPRIMES PELLICULES,6118000041962,,0.0,87.0,0.0,0.0,ETORICOXIB,,1,0
6110,6111,DOLIFEN 400 MG COMPRIMES,30 COMPRIMES PELLICULES,6118000042174,,0.0,27.0,0.0,0.0,IBUPROFENE,,1,0
6111,6112,DOLIFENE 600 MG COMPRIMES,30 COMPRIMES PELLICULES,6118000042181,,0.0,27.0,0.0,0.0,IBUPROFENE,,2,0
6112,6113,DOLTRAM COMPRIMES,20 COMPRIMES PELLICULES,6118000041559,,0.0,27.0,0.0,0.0,PARACETAMOL / TRAMADOL,,2,0


In [70]:
import json

In [66]:
def get_medicament_codeBarre(code_barre):
        url = f"https://medicament.ma/?choice=barcode&s={code_barre}"
        temps_sleep=1
        options = Options()
        options.add_argument("--headless")  # mode sans interface
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        # Initialisation du navigateur
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(temps_sleep)

        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, "html.parser")
        data = {}
        h3_tag = soup.find("h3")
        data["Code_EAN_13"] = code_barre if code_barre else None
        data["Nom"] = h3_tag.get_text(strip=True) if h3_tag else None
        img_tag = soup.select_one("div.only-mobile img")
        data["Image URL"] = img_tag["src"] if img_tag else None
        rows = soup.select("table.table-details tr")
        for row in rows:
            field = row.find("td", class_="field")
            value = row.find("td", class_="value")
            if field and value:
                key = field.get_text(strip=True)
                val = value.get_text(strip=True)
                data[key] = val

        def parse_prix(prix_str):
            """Convertit une chaîne de type '36.20 dhs' en float 36.20"""
            if prix_str:
                return float(prix_str.replace("dhs", "").strip())
            return None

        data["PPV"] = parse_prix(data.get("PPV"))
        data["Prix hospitalier"] = parse_prix(data.get("Prix hospitalier"))
        data["url"] = url
        return data

In [72]:
for i in tqdm.tqdm(resultat['Code_EAN_13']): 
    results = get_medicament_codeBarre(i)
    if results['Nom'] != 'Page non trouvée.':
        with open(f"/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/medicaments/medicaments_{i}.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        print(f"Data for {i} saved successfully.")
    
    

Data for 6118000120568 saved successfully.


 92%|█████████▏| 5649/6114 [3:12:45<19:59,  2.58s/it]

Data for 6118001070961 saved successfully.


 92%|█████████▏| 5651/6114 [3:12:49<17:49,  2.31s/it]

Data for 6118000032830 saved successfully.


...

100%|█████████▉| 6112/6114 [3:30:09<00:04,  2.18s/it]

Data for 6118000042181 saved successfully.


100%|█████████▉| 6113/6114 [3:30:11<00:02,  2.15s/it]

Data for 6118000041559 saved successfully.


100%|██████████| 6114/6114 [3:30:13<00:00,  2.06s/it]


In [24]:

import pymysql

host = "srv1905.hstgr.io"
user = "u791384722_pharma"
password = "@sCM^n4L"
database = "u791384722_pharma"

conn = pymysql.connect(
                    host=host, user=user, password=password, database=database
                )          

In [25]:
cursor = conn.cursor(pymysql.cursors.DictCursor)
request = """INSERT INTO Medicament (
                    Code_EAN_13,
                    Nom,
                    Image_URL,
                    Présentation,
                    Dosage,
                    Distributeur_ou_fabriquant,
                    Composition,
                    Classe_thérapeutique,
                    Statut,
                    Code_ATC,
                    PPV,
                    Prix_hospitalier,
                    Tableau,
                    Indications,
                    Min_Stock,
                    Stock_Actuel, 
                    url_medicament
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                );"""


In [ ]:
cursor.execute(
    request,
    (
        Code_EAN_13,
        Nom,
        Image_URL,
        Présentation,
        Dosage,
        Distributeur_ou_fabriquant,
        Composition,
        Classe_thérapeutique,
        Statut,
        Code_ATC,
        PPV,
        Prix_hospitalier,
        Tableau,
        Indications,
        Min_Stock,
        Stock_Actuel,
        url_medicament,
    ),
)
conn.commit()

In [26]:
import os
import json

def parcourir_dossier_json(chemin_dossier):
    list_of_data = []
    for fichier in os.listdir(chemin_dossier):
        if fichier.endswith(".json"):  # on ne prend que les JSON
            chemin_fichier = os.path.join(chemin_dossier, fichier)
            with open(chemin_fichier, "r", encoding="utf-8") as f:
                try:
                    values = json.load(f) 
                    Code_EAN_13 = values.get("Code_EAN_13")
                    Nom = values.get("Nom")
                    Image_URL = values.get("Image URL")
                    Présentation = values.get("Présentation")
                    Dosage = values.get("Dosage")
                    Distributeur_ou_fabriquant = values.get("Distributeur ou fabriquant")
                    Composition = values.get("Composition")
                    Classe_thérapeutique = values.get("Classe thérapeutique")
                    Statut = values.get("Statut")
                    Code_ATC = values.get("Code ATC")
                    PPV = values.get("PPV")
                    Prix_hospitalier = values.get("Prix hospitalier")
                    Tableau = values.get("Tableau")
                    Indications = values.get("Indication(s)")
                    Min_Stock = 0
                    Stock_Actuel = 0
                    url_medicament = values.get("url")
                    list_of_data.append(values)
                except json.JSONDecodeError:
                    print(f"Erreur : {fichier} n'est pas un JSON valide.")
    return pd.DataFrame(list_of_data)

# Exemple d'utilisation
data_scraped = parcourir_dossier_json("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/medicaments")

In [27]:
data_scraped = data_scraped[["Code_EAN_13", "Nom", "Image URL", "Présentation", "Dosage", "Distributeur ou fabriquant","Composition", "Classe thérapeutique",  "Statut","Code ATC"
                             , "PPV", "Prix hospitalier", "Tableau", "Indication(s)", "url"]]

In [28]:
data_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2670 entries, 0 to 2669
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Code_EAN_13                 2670 non-null   object 
 1   Nom                         2670 non-null   object 
 2   Image URL                   2670 non-null   object 
 3   Présentation                2670 non-null   object 
 4   Dosage                      2499 non-null   object 
 5   Distributeur ou fabriquant  2670 non-null   object 
 6   Composition                 2670 non-null   object 
 7   Classe thérapeutique        2605 non-null   object 
 8   Statut                      2670 non-null   object 
 9   Code ATC                    2626 non-null   object 
 10  PPV                         2658 non-null   float64
 11  Prix hospitalier            2650 non-null   float64
 12  Tableau                     2628 non-null   object 
 13  Indication(s)               2415 

In [47]:
elements_to_add = []
for inde, elem in all_dispo_code['Code_EAN_13'].items(): 
    if elem not in data_scraped['Code_EAN_13'].values: 
        elements_to_add.append(elem)
        

In [48]:
len(elements_to_add)

3439

In [51]:
len(all_dispo_code['Code_EAN_13'].values)

3238